In [1]:
%cd ~/change
import pickle
import torch
import re
import itertools
from transformers import AutoTokenizer, AutoModelForMaskedLM

/home/sibava/change


In [2]:
tokenizer = AutoTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-v2')
model = AutoModelForMaskedLM.from_pretrained('cl-tohoku/bert-base-japanese-v2')

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-v2 were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [34]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32768, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [20]:
with open('data/multclose.pickle',mode='rb') as f:
    train_x = pickle.load(f)
    train_y = pickle.load(f)

In [23]:
kaku_dict ={
    'から':['から','が'],
    'が':['を'],
    'で':['が','で'],
    'と':['と'],
    'に':['が','に'],
    'の':['を'],
    'へ':['へ'],
    'まで':['まで'],
    'を':['を']
    }

In [24]:
kakuid_dict = {}
for k,v in kaku_dict.items():
    kakuid_dict[tokenizer.encode(k,add_special_tokens=False)[0]] = tokenizer.encode(v,add_special_tokens=False)
del kakuid_dict[912]
kakuid_dict[11172] = [11172]
print(kakuid_dict)

{11142: [11142, 862], 862: [932], 889: [862, 889], 890: [890], 893: [862, 893], 896: [932], 906: [906], 932: [932], 11172: [11172]}


In [25]:
mask_train_x = []
mask_train_y = []
for x,y in zip(train_x,train_y):
    # masks = [kaku_dict[x][0] if (len(kaku_dict[x]) == 1) else tokenizer.mask_token for x in l[3]]
    masks = []
    labels = []
    before = []
    for joshi,label in zip(x[3],y):
        if(len(kaku_dict[joshi]) == 1):
            masks.append(kaku_dict[joshi][0]) 
        else:
            masks.append(tokenizer.mask_token)
            labels.append(label)
            before.append(joshi)
    result = [None]*(len(x[1])+len(masks))
    result[::2] = x[1]
    result[1::2] = masks
    mask_train_x.append([''.join(result + [x[2]]),before])
    mask_train_y.append(labels)

In [122]:
a = tokenizer('これはりんごです',return_tensors='pt')
model(a['input_ids'].to(device))

MaskedLMOutput(loss=None, logits=tensor([[[-5.3975,  3.0544, -3.5304,  ..., -6.0975, -3.8352, -2.9906],
         [-5.9088,  5.2070, -5.7163,  ..., -5.2381, -5.2221, -5.0892],
         [-7.4153,  4.3304, -3.6685,  ..., -5.8254, -8.0242, -4.9535],
         [-5.8890,  3.4783, -5.3734,  ..., -3.3031, -3.4347, -4.6271],
         [-6.4066,  2.3010, -3.3480,  ..., -8.6890, -4.8201, -2.0309],
         [-8.6374,  3.0015, -3.7152,  ..., -8.1358, -4.4212, -5.7711]]],
       device='cuda:0', grad_fn=<AddBackward0>), hidden_states=None, attentions=None)

In [114]:
torch.where(a['input_ids'] == tokenizer.sep_token_id)[1]

tensor([5])

In [123]:
def single_mask(x):
    input_ids = tokenizer(x[0],return_tensors='pt') 
    masked_index = torch.where(input_ids['input_ids'] == tokenizer.mask_token_id)[1].tolist()[0]
    input_ids.to(device)
    result = model(input_ids['input_ids'])
    index_list = kakuid_dict[tokenizer.encode(x[1],add_special_tokens=False)[0]]
    candidate_set = [[result[0][:,masked_index][0][i].item(),tokenizer.decode(i).replace(' ','')] for i in index_list]
    candidate_set.sort(reverse=True)   
    return candidate_set[0][1]

In [52]:
i = 0
for x in mask_train_x:
    if (x[0].count('[MASK]') > 1):
        print(x)


['これまで[MASK]資産を民間[MASK]売却する', ['に', 'に']]
['内閣改造[MASK]蕭万長氏を対中担当[MASK]起用する', ['で', 'に']]
['明治神宮[MASK]三が日[MASK]初もうで客を見込む', ['で', 'で']]
['交通博物館[MASK]長坂容疑者[MASK]声を掛ける', ['で', 'に']]
['ガス会社[MASK]間[MASK]実施する', ['で', 'に']]
['一九九四年[MASK]米国[MASK]実用化する', ['から', 'で']]
['米軍ヘリ撃墜事件[MASK]意見を米議会[MASK]出す', ['で', 'で']]
['生活を知らず知らず[MASK]規制[MASK]しばる', ['に', 'で']]
['ほか[MASK]規制[MASK]含む', ['に', 'に']]
['奈良時代[MASK]日本各地[MASK]栽培する', ['に', 'で']]
['十一月[MASK]大阪[MASK]開く', ['に', 'で']]
['秋[MASK]大阪[MASK]開く', ['に', 'で']]
['年始[MASK]至福感[MASK]包むようだ', ['に', 'に']]
['本番[MASK]レギュラー[MASK]選ぶ', ['で', 'に']]
['ゴールライン手前[MASK]相模台工ディフェンス[MASK]追いつく', ['で', 'に']]
['清水平[MASK]凍死状態[MASK]発見する', ['で', 'で']]
['調べ[MASK]乗用車[MASK]追突する', ['で', 'に']]
['五八年[MASK]七三年[MASK]結成する', ['に', 'に']]
['熱戦[MASK]熱気[MASK]包む', ['に', 'に']]
['六日[MASK]官報[MASK]掲載する', ['に', 'に']]
['停戦協定[MASK]要請[MASK]記述を盛り込む', ['に', 'で']]
['軍事進攻問題を１２日[MASK]会合[MASK]取り上げる', ['に', 'で']]
['東京裁判[MASK]Ａ級戦犯[MASK]指名する', ['で', 'に']]
['三四年[MASK]立命館出版部[MASK]刊行する', ['に', 'から']]
['野間宏の会を二時[MASK]日本出版

In [124]:
def mult_mask(x):
    correct = 0
    s = x[0]
    k = x[1].copy()
    t = s.count(tokenizer.mask_token)
    result = []
    if(t == 0):
        return []
    
    indexs = [m.end() for m in re.finditer('\[MASK\]',s)][:-1]
    while(s.count(tokenizer.mask_token)>1):
        i = indexs.pop()
        pred = single_mask([s[i:],k.pop()]) 
        result.append(pred)
        s = s[:i] + s[i:].replace('[MASK]',pred)
    result.append(single_mask([s,k]))       
    return result[::-1]
     

In [125]:
pred = [mult_mask(x) for x in mask_train_x]

In [126]:
def eval(train_x,train_y):
    n = 0
    miss = 0
    for xs,ys in zip(train_x,train_y):
        pred = mult_mask(xs)
        if(len(pred) == 0):
            n += 1
            continue
        for x,y in zip(pred,ys):
            if(x in y):
                n += 1
            else:
                n += 1
                miss += 1
                print(f'{xs[0]},返還前:{xs[1]},変換後:{x},正解:{y}')
    return (1788 - miss) / 1788

In [127]:
eval(mask_train_x,mask_train_y) #0.9273

時[MASK]許す,返還前:['に'],変換後:が,正解:に
海外[MASK]大蔵分割論を唱える,返還前:['から'],変換後:から,正解:が
選挙通[MASK]知る薄れる心配するのだ,返還前:['で'],変換後:が,正解:で
文学[MASK]しばる,返還前:['に'],変換後:に,正解:が
文章[MASK]ひく,返還前:['に'],変換後:に,正解:が
賞[MASK]選ぶ,返還前:['に'],変換後:が,正解:に
メード[MASK]育てる,返還前:['に'],変換後:に,正解:が
活躍[MASK]触発する,返還前:['に'],変換後:に,正解:が
外国勢[MASK]韓国勢を注目する,返還前:['で'],変換後:が,正解:で
一作[MASK]選ぶ,返還前:['に'],変換後:が,正解:に
ノーベル文学賞[MASK]選ぶ,返還前:['に'],変換後:が,正解:に
ＴＢＳ[MASK]開く,返還前:['で'],変換後:が,正解:で
交通博物館[MASK]長坂容疑者[MASK]声を掛ける,返還前:['で', 'に'],変換後:に,正解:が
子育て[MASK]追いまくる,返還前:['に'],変換後:に,正解:が
知人[MASK]囲む,返還前:['に'],変換後:に,正解:が
家族[MASK]反対する,返還前:['に'],変換後:に,正解:が
ベール[MASK]包む,返還前:['に'],変換後:に,正解:が
国[MASK]ザカップを持ち込む,返還前:['に'],変換後:が,正解:に
ベール[MASK]包む,返還前:['に'],変換後:に,正解:が
人模様[MASK]彩る,返還前:['で'],変換後:で,正解:が
頭文字[MASK]つける,返還前:['から'],変換後:が,正解:から
攻撃[MASK]破壊する,返還前:['で'],変換後:で,正解:が
米軍ヘリ撃墜事件[MASK]意見を米議会[MASK]出す,返還前:['で', 'で'],変換後:で,正解:other
海外[MASK]規制解除を求める,返還前:['から'],変換後:から,正解:が
生活を知らず知らず[MASK]規制[MASK]しばる,返還前:['に', 'で'],変換後:で,正解:が
車イス[MASK]最高速度を決める,返還前:['に'],変換後:が,正解:に
所[MASK]起用する,返還前:['に'],変換後:が

0.9200223713646533